# 차원 축소 : PCA

## 0. 들어가기 전에: 왜 차원 축소를 할까요?

### 차원의 저주 (Curse of Dimensionality)
"컬럼(변수)이 하나 추가될 때마다, 우리가 탐색해야 할 공간은 어떻게 변할까요?"
단순한 덧셈이 아닙니다. 곱셈으로 늘어나는 무시무시한 폭발을 확인해보세요.

| 컬럼 수 (차원) | 조합 계산 (구간 10개 가정) | 총 조합의 수 (공간 크기) |
|---|---|---|
| 1개 | 10 | 10개 (직선) |
| 2개 | 10 × 10 | 100개 (평면) |
| 3개 | 10 × 10 × 10 | 1,000개 (입체) |
| 10개 | 10 × ... (10번) | 100억 개 |
| 30개 | 10 × ... (30번) | 10^30 (우주 별 개수 초과!) |

### PCA의 해결책
그래서 우리는 **가장 중요한 정보만 남기고** 차원을 줄입니다.

*   **시각화 가능:** 4차원 이상의 데이터는 눈으로 볼 수 없지만, 2~3차원으로 줄이면 시각화가 가능합니다.
*   **노이즈 제거:** 중요하지 않은 특징(Feature)을 제거하여 모델이 패턴을 더 잘 찾게 도와줍니다.
*   **효율성 증대:** 데이터 크기를 줄여 계산 속도를 높이고 메모리를 절약합니다.

## 1.환경준비
### (1) 라이브러리 로딩

In [ ]:
!pip install plotly -q
import plotly.graph_objects as go

In [ ]:
# 기본 라이브러리 가져오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

from sklearn.datasets import load_breast_cancer, load_digits, load_iris, make_swiss_roll
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA

### (2) 샘플 데이터 생성하기

In [ ]:
# 럭비공 형태의 샘플 데이터 생성 함수
def generate_rugby_data(n_points=1000, a=1, b=1.5, c=2):
    phi = np.random.uniform(0, np.pi, n_points)
    theta = np.random.uniform(0, 2*np.pi, n_points)
    x = a * np.sin(phi) * np.cos(theta)
    y = b * np.sin(phi) * np.sin(theta)
    z = c * np.cos(phi)
    X = np.column_stack((x, y, z))
    return X

rugby = generate_rugby_data()

# 스위스롤 데이터
swiss_roll, _ = make_swiss_roll(n_samples=1000, noise=0.2)

In [ ]:
# 3차원 스캐터 함수 생성
def my_3d_Scatter(X) :
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2],
                           mode='markers', marker=dict(size=2, color='blue'),
                           name='Original Data'))

    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                      scene=dict(xaxis_title='X Axis', yaxis_title='Y Axis', zaxis_title='Z Axis'))

    fig.show()

## 2.PCA 개념이해

### (1) 럭비공 형태의 데이터 차원 축소
*   원본 데이터 둘러보기

In [ ]:
my_3d_Scatter(rugby)

*   차원 축소

In [ ]:
# PCA를 이용하여 2개의 주성분으로 차원 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(rugby)

# PCA 축소 데이터 조회
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.grid()
plt.show()

### (2) 스위스롤 형태의 데이터 차원 축소
*   원본 데이터 둘러보기

In [ ]:
my_3d_Scatter(swiss_roll)

*   차원 축소

In [ ]:
# PCA를 이용하여 2개의 주성분으로 차원 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(swiss_roll)

# PCA 축소 데이터 조회
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.grid()
plt.show()

## 3.PCA 사용해보기
### (1) 데이터 준비
*   데이터 로딩

In [ ]:
iris = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/iris.csv")
target = 'Species'
x = iris.drop(target, axis = 1)
y = iris.loc[:, target]

In [ ]:
x.head()

*   스케일링

In [ ]:
scaler = MinMaxScaler()
x2 = scaler.fit_transform(x)

# (옵션)데이터프레임 변환
x2 = pd.DataFrame(x2, columns= x.columns)

### (2) 주성분 분석

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# feature 수
x2.shape[1]

*   주 성분 분석 수행

In [ ]:
# 주성분 수 2개
n = 2
pca = PCA(n_components = n)

# 만들고, 적용하기(결과는 넘파이 어레이)
x2_pc = pca.fit_transform(x2)

In [ ]:
# 2개의 주성분
x2_pc[:5]

In [ ]:
# (옵션) 데이터프레임으로 변환
x2_pc = pd.DataFrame(x2_pc, columns = ['PC1', 'PC2'])
x2_pc.head()

*   기존 데이터에 차원 축소된 데이터를 붙여 봅시다.

In [ ]:
pd.concat([iris, x2_pc], axis = 1).head()

*   두개의 주성분 시각화

In [ ]:
sns.scatterplot(x = 'PC1', y = 'PC2', data = x2_pc, hue = y)
plt.grid()
plt.show()

## 4.고차원 데이터 차원축소
### (1) 데이터 준비
#### 1) 데이터 로딩

In [ ]:
# breast_cancer 데이터 로딩
cancer=load_breast_cancer()
x = cancer.data
y = cancer.target

x = pd.DataFrame(x, columns=cancer.feature_names)

x.shape

In [ ]:
x.head()

In [ ]:
x.info()

In [ ]:
x.describe().T

#### 2) 스케일링
*   거리계산 기반 차원축소이므로 스케일링 필요

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

# (옵션)데이터프레임 변환
x = pd.DataFrame(x, columns=cancer.feature_names)

#### 3) 데이터 분할
*   train, validation 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (2) 주성분 만들기

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# feature 수
x_train.shape[1]

*   주 성분 분석 수행

In [ ]:
# 주성분을 몇개로 할지 결정(최대값 : 전체 feature 수)
n = x_train.shape[1]

# 주성분 분석 선언
pca = PCA(n_components=n)

# 만들고, 적용하기
x_train_pc = pca.fit_transform(x_train)
x_val_pc = pca.transform(x_val)

*   편리하게 사용하기 위해 데이터프레임으로 변환

In [ ]:
# 칼럼이름 생성
column_names = [ 'PC'+str(i+1) for i in range(n) ]
column_names

In [ ]:
# 데이터프레임으로 변환하기
x_train_pc = pd.DataFrame(x_train_pc, columns = column_names)
x_val_pc = pd.DataFrame(x_val_pc, columns = column_names)
x_train_pc

*   문1) 다음의 조건으로 주성분을 추출해 봅시다.
*   주성분 1개로 선언하고, x_train을 이용해서 주성분 추출
*   주성분 2개로 선언하고, x_train을 이용해서 주성분 추출
*   주성분 3개로 선언하고, x_train을 이용해서 주성분 추출

In [ ]:
# 주성분 1개짜리
pca1 = PCA(n_components=1)
x_pc1 = pca1.fit_transform(x_train)

In [ ]:
# 주성분 2개짜리
pca2 = PCA(n_components=2)
x_pc2= pca2.fit_transform(x_train)

In [ ]:
# 주성분 3개짜리
pca3 = PCA(n_components=3)
x_pc3= pca3.fit_transform(x_train)

*   문2) 각 주성분 결과에서 상위 3개 행씩 조회하여 비교해 봅시다.

In [ ]:
print(x_pc1[:3])
print('-'*50)
print(x_pc2[:3])
print('-'*50)
print(x_pc3[:3])

### (3) 주성분 누적 분산 그래프
*   그래프를 보고 적절한 주성분의 개수를 지정(elbow method!)
*   x축 : PC 수
*   y축 : 전체 분산크기 - 누적 분산크기

In [ ]:
plt.plot(range(1,n+1), pca.explained_variance_ratio_, marker = '.')
plt.xlabel('No. of PC')
plt.grid()
plt.show()

주성분 개수 몇개면 충분할까요?

### (4) 시각화
*   주 성분 중 상위 2개를 뽑아 시각화 해 봅시다.

In [ ]:
sns.scatterplot(x = 'PC1', y = 'PC2', data = x_train_pc, hue = y_train)
plt.grid()
plt.show()

# 5.복습
오늘 수업 복습은, 별도 문제가 주어지지 않고, 배운 내용을 다시 살펴보고 코드를 실행해보면서 직접 Review하는 것입니다.^^